# Data Preprocessing and Cleaning

Data cleaning is an essential step in the process of preparing data for further processing and serving into a large language model. Without data cleaning, the language model would be limited in its ability to interpret and process the data accurately. Data cleaning helps to ensure that the data is in a consistent format, free of any inconsistencies or errors that could lead to incorrect results. It also helps to reduce the amount of time needed to process the data, as it eliminates the need to manually check for any inconsistencies or errors. Additionally, data cleaning can help to improve the accuracy of the language model by removing any irrelevant or incorrect data. By taking the time to properly clean the data, the language model can more accurately interpret and process the data, leading to better results.

> 📍 Fill out the missing pieces in the source source to get everything working (indicated by `#FIXME`).

## Count tokens

> **Important note**
>
> *You do not need to manually tokenize strings before feeding texts into the model. This will be done automatically once you put instructions into the `prompt` parameter. However, you can use the `tiktoken` library to check how a string is tokenized and count the numbers of tokens to calculate the cost of an API call. Learn more [here](https://platform.openai.com/docs/introduction/tokens).*

Tokenizing text strings is an important step in natural language processing (NLP) as it helps models like GPT-3 understand the structure of a text string. Tokenizing breaks a text string into smaller pieces called tokens, which can then be analyzed and used by the model. By understanding the structure of a text string, models can better understand the meaning of the text. Additionally, tokenizing helps to determine the cost of an Azure OpenAI Service API call, as usage is priced by token. Furthermore, different models use different encodings, so it is important to tokenize text strings in the appropriate format.

`tiktoken` supports three encodings used by Azure OpenAI Service models:

| Encoding name | Azure OpenAI Service models |
| ------------- | -------------- |
| gpt2 (or r50k_base) | Most GPT-3 models |
| p50k_base | Code models, text-davinci-002, text-davinci-003 |
| cl100k_base | text-embedding-ada-002 |

Tokens in English typically range from one character to one word (e.g. "t" or "great"), though some languages may have tokens that are shorter or longer than one character or word. Spaces are usually placed at the beginning of words (e.g. " is" instead of "is " or "+"is"). You can use the Tokenizer to quickly check how a string is tokenized.

To show it briefly, we will use `tiktoken` to tokenize a text string and see how the output looks like.

In [0]:
%pip install tiktoken

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import tiktoken

encoding = tiktoken.get_encoding("p50k_base")
encoding.encode("Hello world, this is fun!")

Out[8]: [15496, 995, 11, 428, 318, 1257, 0]

Write a script that shows the string tokens from an input phrase.

In [0]:
[encoding.decode_single_token_bytes(token) for token in encoding.encode("Hello world, this is fun!")]


Out[9]: [b'Hello', b' world', b',', b' this', b' is', b' fun', b'!']

Let's write a function to count the number of tokens in a text string.

In [0]:
def get_num_tokens_from_string(string: str, encoding_name: str='p50k_base') -> int:
    """Returns the number of tokens in a text by a given encoding."""
    encoding = tiktoken.get_encoding(encoding_name)
    return len(encoding.encode(string))

get_num_tokens_from_string("Hello World, this is fun!")

Out[3]: 7

## Clean data

Next we'll perform some light data cleaning by removing redundant whitespace and cleaning up the punctuation to prepare the data for tokenization. Again, tokenization is not required for the model to work, but it is a good practice to do so to ensure that the data is in a consistent format and that the model is able to process the data correctly. Also, it makes sure that the request is not too long for the model as the maximum number of tokens for davinci is 2048, e.g., equivalent to around 2-3 pages of text.

> **Best Practices**
> - **Replace newlines with a single space**: Unless you're embedding code, we suggest replacing newlines (\n) in your input with a single space, as we have observed inferior results when newlines are present.

In [0]:
import re

def normalize_text(string, sep_token = " \n "):
    """Normalize text by removing unnecessary characters and altering the format of words."""
    # make text lowercase
    string = re.sub(r'\s+',  ' ', string).strip()
    string = re.sub(r". ,","",string)
    # remove all instances of multiple spaces
    string = string.replace("..",".")
    string = string.replace(". .",".")
    string = string.replace("\n", "")
    string = string.strip()
    return string

Generate some data to test the cleaning function.

In [0]:
text = "Hola buenos días.. ¿Qué \n tal está usted?"
normalize_text(text)

Out[13]: 'Hola buenos días. ¿Qué tal está usted?'

## Ejercicio 1: Validar longitud de texto antes de enviar a OpenAI

Escribe una función llamada validate_text_length que valide si un texto supera un límite de tokens especificado. Si el texto supera el límite, la función debe dividirlo en segmentos de tamaño adecuado para enviarlos al modelo de OpenAI. Usa la codificación cl100k_base.



In [0]:
import tiktoken

def validate_text_length(text, max_tokens=2048, encoding_name='cl100k_base'):
    """
    Divide un texto en segmentos que no excedan el número máximo de tokens permitidos.

    Args:
        text (str): El texto a dividir.
        max_tokens (int): El número máximo de tokens por segmento.
        encoding_name (str): El nombre de la codificación para calcular tokens.

    Returns:
        list: Una lista de segmentos de texto.
    """
    # Obtén el codificador
    encoding = tiktoken.get_encoding(encoding_name)
    
    # Convierte el texto en una lista de tokens
    tokens = encoding.encode(text)
    
    # Divide los tokens en segmentos de longitud máxima
    segments = []
    for i in range(0, len(tokens), max_tokens):
        segment_tokens = tokens[i:i + max_tokens]
        # Decodifica los tokens de vuelta a texto
        segment_text = encoding.decode(segment_tokens)
        segments.append(segment_text)
    
    return segments

# Prueba la función
text = "Lorem ipsum " * 500  # Texto largo
segments = validate_text_length(text)

# Verifica el número de segmentos y su longitud
print(len(segments))  # Resultado esperado: Más de 1 segmento
print(len(tiktoken.get_encoding('cl100k_base').encode(segments[0])))  # <= XXX tokens
##Resultado esperado:

##El texto se divide en segmentos adecuados para ser procesados por OpenAI.
##Cada segmento tendrá un número de tokens menor o igual al límite especificado.


1
1001


## Ejercicio 2: Filtrar palabras prohibidas en un texto

Escribe una función llamada filter_prohibited_words que remueva palabras prohibidas de un texto antes de enviarlo a OpenAI. 

Como ejemplo queremos Eliminar palabras como: ['password', 'confidential', 'secret']. 



In [0]:
import re

def filter_prohibited_words(text, prohibited_words):
    """
    Remueve palabras prohibidas de un texto, reemplazándolas por [REDACTED].

    Args:
        text (str): El texto original.
        prohibited_words (list): Lista de palabras prohibidas.

    Returns:
        str: El texto con las palabras prohibidas reemplazadas por [REDACTED].
    """
    # Construir una expresión regular para las palabras prohibidas
    pattern = r'\b(' + '|'.join(re.escape(word) for word in prohibited_words) + r')\b'
    
    # Reemplazar palabras prohibidas con [REDACTED]
    cleaned_text = re.sub(pattern, '[REDACTED]', text, flags=re.IGNORECASE)
    
    return cleaned_text

# Prueba la función
text = "This document contains confidential and secret information."
prohibited_words = ['password', 'confidential', 'secret']

cleaned_text = filter_prohibited_words(text, prohibited_words)
print(cleaned_text)  # Resultado esperado: "This document contains [REDACTED] and [REDACTED] information."


This document contains [REDACTED] and [REDACTED] information.


## Ejercicio 3: Identificar idioma del texto

Crea una función detect_language que use langdetect para identificar el idioma de un texto y devuelva el código del idioma (e.g., en, es). Verifica el idioma antes de enviarlo al modelo OpenAI. Debes instalar la libreria langdetect


In [0]:
%pip install langdetect

Python interpreter will be restarted.
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=7093064f3858ab069849c32348b6c271d71a30353b6b6d99a23e29703afb5af9
  Stored in directory: /root/.cache/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect
Python interpreter will be restarted.


In [0]:
from langdetect import detect

def detect_language(text):
    """
    Detecta el idioma de un texto y devuelve el código del idioma.
    
    Args:
        text (str): El texto a analizar.
        
    Returns:
        str: Código del idioma detectado (e.g., 'en' para inglés, 'es' para español).
    """
    try:
        # Detectar el idioma del texto
        language_code = detect(text)
        return language_code
    except Exception as e:
        # Manejar posibles errores (e.g., texto vacío)
        return f"Error detectando idioma: {str(e)}"

# Prueba la función
text = "Hola, este es un texto en español."
language = detect_language(text)
print(language)  # Resultado esperado: 'es' (para español)


es


## Ejercicio 5: Generar prompts efectivos

Escribe una función generate_effective_prompt que genere un prompt optimizado para OpenAI, dado un tema y un objetivo. Usa plantillas como:
“Explain [tema] in simple terms for a [audiencia].”



In [0]:
def generate_effective_prompt(topic, audience="general audience"):
    """
    Genera un prompt optimizado para OpenAI dado un tema y una audiencia.

    Args:
        topic (str): El tema a tratar.
        audience (str): La audiencia objetivo. Por defecto es "general audience".

    Returns:
        str: Un prompt bien estructurado para OpenAI.
    """
    # Crear la plantilla del prompt
    prompt = f"Explain {topic} in simple terms for a {audience}."
    return prompt

# Prueba la función
topic = "machine learning"
audience = "beginner"
prompt = generate_effective_prompt(topic, audience)
print(prompt)  # Resultado esperado: "Explain machine learning in simple terms for a beginner."


Explain machine learning in simple terms for a beginner.


## Ejercicio 6: Medir costo estimado del uso de OpenAI

Crea una función estimate_cost que calcule el costo estimado de una solicitud a OpenAI basado en el número de tokens. Supón un costo de $0.02 por 1000 tokens.



In [0]:
import tiktoken

def estimate_cost(text, encoding_name='cl100k_base', cost_per_1k_tokens=0.02):
    """
    Calcula el costo estimado de una solicitud a OpenAI basado en el número de tokens.

    Args:
        text (str): El texto a evaluar.
        encoding_name (str): El nombre de la codificación (por defecto 'cl100k_base').
        cost_per_1k_tokens (float): Costo por 1000 tokens (por defecto $0.02).

    Returns:
        float: El costo estimado en dólares.
    """
    # Obtener la codificación
    encoding = tiktoken.get_encoding(encoding_name)
    
    # Codificar el texto y contar los tokens
    token_count = len(encoding.encode(text))
    
    # Calcular el costo
    cost = (token_count / 1000) * cost_per_1k_tokens
    
    return round(cost, 6)  # Redondear a 6 decimales

# Prueba la función
text = "This is a sample text." * 100
cost = estimate_cost(text)
print(cost)  # Resultado esperado: Costo aproximado basado en la cantidad de tokens


0.01002


## Ejercicio 7: Limpiar texto JSON para OpenAI

Escribe una función clean_json_text que tome un JSON como entrada y limpie todas las claves y valores de caracteres no ASCII.


In [0]:
import json
import re

def clean_value(value):
    """Limpia un valor eliminando caracteres no ASCII."""
    # Usar una expresión regular para eliminar cualquier carácter no ASCII
    return re.sub(r'[^\x00-\x7F]+', '', value)

def clean_json_text(json_obj):
    """Limpia texto JSON eliminando caracteres no ASCII en claves y valores."""
    # Limpiar tanto las claves como los valores del JSON
    cleaned_json = {}
    
    for k, v in json_obj.items():
        # Limpiar clave y valor
        cleaned_key = clean_value(k)
        cleaned_value = clean_value(v)
        cleaned_json[cleaned_key] = cleaned_value
        
    return cleaned_json

# Prueba la función
data = {"key1": "Hello 😊", "key2": "Café and thé"}
cleaned_data = clean_json_text(data)
print(cleaned_data)  # Resultado esperado: {"key1": "Hello ", "key2": "Caf and th"}


{'key1': 'Hello ', 'key2': 'Caf and th'}
